In [5]:
from src.Get_data_DB import DataTransformer
import pandas as pd
# Tạo instance của class
transformer = DataTransformer()

# Lấy dữ liệu lần đầu từ năm 2024 Mysql
mysql_query = """SELECT 
                        tp.id AS Id,
                        t.money Doanh_thu,
                        s.money AS Doanh_so,
                        DATE_ADD(t.payment_at, INTERVAL 7 HOUR) AS Ngay_chuyen_tien,
                        s.email as Khach_hang,
                        p.id AS Ma_khoa_hoc,
                        o.current_sale_id AS Ma_saler,
                        l.utm_medium AS Ma_marketer,
                        o.source AS Ma_kenh
    FROM transactions t
    JOIN transactions_products tp ON tp.p_transaction_id = t.id
    JOIN products p ON tp.product_id = p.id
    JOIN orders o ON t.order_id = o.id
    join students s on s.order_id=o.id
    JOIN leads l ON o.lead_id = l.id
    join users u on o.current_sale_id=u.id
    WHERE t.t_status = 1
      AND DATE_ADD(t.payment_at, INTERVAL 7 HOUR) >= '2024-01-01'
      And u.role=3
    group by Id, Doanh_thu, Doanh_so, Ngay_chuyen_tien, Khach_hang, Ma_khoa_hoc, Ma_saler, Ma_marketer, Ma_kenh
                    """
df = transformer.fetch_from_mysql(mysql_query)

In [6]:
df = df.fillna(0)
df['Ngay_chuyen_tien'] = pd.to_datetime(df['Ngay_chuyen_tien'])

In [7]:
df

,Id,Doanh_thu,Doanh_so,Ngay_chuyen_tien,Khach_hang,Ma_khoa_hoc,Ma_saler,Ma_marketer,Ma_kenh
0,14461,1000000,2392000,2024-01-01 07:00:00,drhahungtb121676@gmail.com,528,85,ADMIN,4
1,14463,6933217,6933217,2024-01-03 07:00:00,Daihongphat2017@gmail.com,529,75,admin,1
2,14464,2392000,2392000,2024-01-04 07:00:00,luongvanhopvn@gmail.com,528,65,Hotline,0
3,14465,16920000,16920000,2024-01-04 07:00:00,tuanminhh1710@gmail.com,515,85,tự nhiên,0
4,14467,5062000,5062000,2024-01-22 07:00:00,nhung.h.hoang@gmail.com,512,75,,0
...,...,...,...,...,...,...,...,...,...
1135,15756,2000000,12000000,2025-07-21 00:00:00,ly.chuminh@gmail.com,529,113,DangHang,0
1136,15757,23000000,25000000,2025-06-26 00:00:00,lyhoang0104ls@gmail.com,553,113,HienPT,0
1137,15758,8500000,8500000,2025-07-21 00:00:00,Ductranhuylund@gmail.com,529,75,BINHND,0
1138,15759,6000000,12000000,2025-07-22 00:00:00,nghtri2025@gmail.com,529,75,,0


In [8]:
def dieu_kien(group):
    if len(group) > 1:
        tong = group['Doanh_thu'].sum()
        mot_gia_tri = group['Doanh_so'].iloc[0]  # Hoặc chọn logic khác
        return tong > mot_gia_tri
    return False

df_lap = (
    df.groupby(['Khach_hang', 'Ma_khoa_hoc'])
      .filter(dieu_kien)
      .sort_values(['Khach_hang', 'Ma_khoa_hoc', 'Ngay_chuyen_tien'])
)

df_lap.to_excel("/tmp/Chek.xlsx", index=False)


---

In [1]:
from src.Get_data_DB import DataTransformer
import pandas as pd
# Tạo instance của class
transformer = DataTransformer()

# Lấy dữ liệu lần đầu từ năm 2024 Mysql
mysql_query = """select code
from offline_classes 
where date(open_at)>= '2024-01-01' and lessons_num =0
                    """
df = transformer.fetch_from_mysql(mysql_query)
df.to_excel("/tmp/Danh sách lớp.xlsx",index=False)